In [ ]:
import tensorflow as tf

import numpy as np
import pandas as pd
import IPython.display as display

In [ ]:
TEST_FOLD = 1

In [ ]:
IS_UPSAMPLED = True

In [ ]:
user = "x"
if user == "x":
    dpath = ""

if IS_UPSAMPLED:
    pref = "upsampled_"
else:
    pref = ""

data = pd.read_csv(dpath + pref + "grouped_for_tf.csv")
train_save_path = dpath + pref + "train_fold_" + str(TEST_FOLD) + ".tfrecord"
test_save_path = dpath + pref + "test_fold_" + str(TEST_FOLD) + ".tfrecord"
data.head()

In [ ]:
test_partition = data[data.Fold == TEST_FOLD].sample(frac=1).reset_index(drop=True)
train_partition = data[data.Fold != TEST_FOLD].sample(frac=1).reset_index(drop=True)

In [ ]:
test_partition

In [ ]:
# The following functions can be used to convert a value to a type compatible
# with tf.Example.

def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


In [ ]:
def serialize_example(split_id, sentence, label):
    """
    Creates a tf.Example message ready to be written to a file.
    """
    # Create a dictionary mapping the feature name to the tf.Example-compatible
    # data type.
    feature = {
      'idx': _int64_feature(split_id),
      'sentence': _bytes_feature(sentence),
      'label': _int64_feature(label)
    }

    # Create a Features message using tf.train.Example.

    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()


In [ ]:
serialized_example = serialize_example(3, b"han er en klar kat", 2)
serialized_example

In [ ]:
example_proto = tf.train.Example.FromString(serialized_example)
example_proto

In [ ]:
def tf_serialize_example(split_id, sentence, label):
    tf_string = tf.py_function(
        serialize_example,
        (split_id, sentence, label),  # pass these args to the above function.
        tf.string)      # the return type is `tf.string`.
    return tf.reshape(tf_string, ()) # The result is a scalar


In [ ]:
def create_tf_dataset(split_id, sentences, labels):
    features_dataset = tf.data.Dataset.from_tensor_slices((split_id, sentences, labels))
    serialized_features_dataset = features_dataset.map(tf_serialize_example)
    return serialized_features_dataset


def write_tf_records(dat, filename = 'test.tfrecord'):
    """
    dat: serialized_features_dataset
    """
    writer = tf.data.experimental.TFRecordWriter(filename)
    writer.write(dat)


In [ ]:
np.unique(test_partition["Diagnosis"])

In [ ]:
# Not currently used
label_map = {"Control":0,
             "Depression":1,
             "Schizophrenia":2}

In [ ]:
# Test set
def from_df_to_tfdata(df):
    sentences = [str(t) for t in df["Transcript.Split"]]
    splt_ids = [int(i) for i in df["Split.ID"]]
    labs = [label_map[d] for d in df["Diagnosis"]]

    tf_data = create_tf_dataset(split_id = splt_ids,
                                sentences = sentences,
                                labels = labs)
    return tf_data



In [ ]:
tf_test_data = from_df_to_tfdata(test_partition)
tf_train_data = from_df_to_tfdata(train_partition)

In [ ]:
tf_test_data

In [ ]:
tf_train_data

In [ ]:
write_tf_records(tf_test_data, test_save_path)
write_tf_records(tf_train_data, train_save_path)